# KAFKA PRODUCER

In [4]:
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
import tweepy
import os
import json
from kafka import KafkaProducer

ACCESS_TOKEN="1052919765126316033-eATAf2skyDxIomK4GxYaZ7zj5Th6BX"
ACCESS_TOKEN_SECRET="fMFNeiSTpMu4kJuZx0vXYgKeizRGv8CZZQaZ2LbY576lH"
CONSUMER_KEY="0wjVKNcrlxQIlf0UKVNwpwShq"
CONSUMER_SECRET="mUoz9LP810L2bkJzfpM1e8TgK2wLmip3kc7nORjvAbwwmtKxid"

SEARCH_KEYWORD = str(input("Please enter a keyword to filter\n"))

producer = KafkaProducer(bootstrap_servers=['kafka:9092'])

def filter_tweet(tweet):
    data = json.loads(tweet)
    
    if 'delete' in data:
        return False
    if 'timestamp_ms' not in data or 'entities' not in data:
        return False      
    if len(data['entities']['hashtags']) <= 0:
        return False
        
    data = {"timestamp": data['timestamp_ms'], "hashtags": [i["text"] for i in data['entities']['hashtags']]}
    print(data)
    data = json.dumps(data)
    
    return data


class StdOutListener(StreamListener):
    def __init__(self):
        self.count=0

    def on_data(self, data):
        data = filter_tweet(data)
        if data != False:
            producer.send("tweets", data.encode('utf-8'))  
         
        self.count+=1 
        print(self.count)
        return True
    def on_error(self, status):
        print(status)
        
    def on_exception(self, exception):
        print(exception)
        return

myStreamListener = StdOutListener()
auth = OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
auth.set_access_token(ACCESS_TOKEN, ACCESS_TOKEN_SECRET)
myStream = tweepy.Stream(auth = auth, listener=myStreamListener)
myStream.filter(track=[SEARCH_KEYWORD])

print(f"Starting tweet streams")

Please enter a keyword to filter
EURO2020
1
{'timestamp': '1624615945252', 'hashtags': ['Euro2020']}
2
{'timestamp': '1624615946213', 'hashtags': ['EURO2020']}
3
4
{'timestamp': '1624615946850', 'hashtags': ['EURO2020']}
5
6
7
8
9
{'timestamp': '1624615948106', 'hashtags': ['EURO2020', 'ENG']}
10
{'timestamp': '1624615948139', 'hashtags': ['EUtweets']}
11
12
{'timestamp': '1624615948851', 'hashtags': ['euro2020', 'europhoria', 'roundof16']}
13
14
{'timestamp': '1624615949402', 'hashtags': ['EURO2020']}
15
{'timestamp': '1624615950202', 'hashtags': ['EURO2020', 'WhistlePodu', 'Yellove', 'kadaikutty']}
16
{'timestamp': '1624615950701', 'hashtags': ['TousEnsemble', 'EURO2020']}
17
18
{'timestamp': '1624615951153', 'hashtags': ['ENG', 'GER']}
19
20
{'timestamp': '1624615952150', 'hashtags': ['OTD']}
21
{'timestamp': '1624615953608', 'hashtags': ['Euro2020', 'Euro2020OnDStv']}
22
{'timestamp': '1624615953745', 'hashtags': ['EURO2020']}
23
24
25
26
{'timestamp': '1624615955015', 'hashtags': 

203
{'timestamp': '1624616055603', 'hashtags': ['ハイセンス']}
204
205
{'timestamp': '1624616055881', 'hashtags': ['OnThisDay', 'EURO2020']}
206
{'timestamp': '1624616056514', 'hashtags': ['BK', 'TeamBK', 'Soccer', 'Football']}
207
{'timestamp': '1624616056687', 'hashtags': ['EURO2020']}
208
{'timestamp': '1624616058283', 'hashtags': ['ENG', 'GER']}
209
210
211
212
213
{'timestamp': '1624616061307', 'hashtags': ['EURO2020']}
214
215
{'timestamp': '1624616061971', 'hashtags': ['EURO2020']}
216
{'timestamp': '1624616061974', 'hashtags': ['EUROSkills', 'EURO2020']}
217
{'timestamp': '1624616062280', 'hashtags': ['EURO2020']}
218
219
220
221
{'timestamp': '1624616064515', 'hashtags': ['ENG', 'mufc', 'Euro2020']}
222
{'timestamp': '1624616064931', 'hashtags': ['EURO2020']}
223
{'timestamp': '1624616064829', 'hashtags': ['EURO2020']}
224
225
{'timestamp': '1624616066452', 'hashtags': ['EURO2020', 'r7ms431']}
226
227
228
229
230
{'timestamp': '1624616069169', 'hashtags': ['Delay', 'JanDelay', 'EUR

{'timestamp': '1624616163644', 'hashtags': ['XoliMfeka']}
408
409
410
411
{'timestamp': '1624616164068', 'hashtags': ['EURO2020']}
412
{'timestamp': '1624616164301', 'hashtags': ['Euro', 'Internationals', 'Belgium', 'Euro2020', 'H2H']}
413
414
415
416
417
{'timestamp': '1624616165095', 'hashtags': ['FRA', 'EURO2020']}
418
419
{'timestamp': '1624616165854', 'hashtags': ['ManCity', 'EURO2020']}
420
421
{'timestamp': '1624616166521', 'hashtags': ['EURO2020']}
422
423
{'timestamp': '1624616167288', 'hashtags': ['EURO2020']}
424
{'timestamp': '1624616167756', 'hashtags': ['EURO2020']}
425
426
{'timestamp': '1624616168582', 'hashtags': ['EURO2020']}
427
{'timestamp': '1624616169019', 'hashtags': ['EURO2020']}
428
429
{'timestamp': '1624616169224', 'hashtags': ['cmoncymru', 'TogetherStronger', 'cymru', 'WAL', 'EURO2020']}
430
{'timestamp': '1624616169228', 'hashtags': ['CmonCymru', 'HenWladFyNhadau', 'EURO2020', 'WAL']}
431
432
{'timestamp': '1624616170016', 'hashtags': ['EURO2020', 'FRA', 'P

{'timestamp': '1624616264761', 'hashtags': ['OTD', 'EURO2020', 'FRA']}
595
{'timestamp': '1624616264926', 'hashtags': ['Giveaways', 'Concours']}
596
{'timestamp': '1624616265548', 'hashtags': ['EURO2020']}
597
598
{'timestamp': '1624616266257', 'hashtags': ['PSG', 'beuh']}
599
600
{'timestamp': '1624616267022', 'hashtags': ['BigMeteorGame', 'EURO21']}
601
{'timestamp': '1624616267431', 'hashtags': ['EURO2020']}
602
603
{'timestamp': '1624616269277', 'hashtags': ['bbcaq', 'EURO2020']}
604
{'timestamp': '1624616269356', 'hashtags': ['EURO2020', 'WhistlePodu', 'Yellove', 'kadaikutty']}
605
606
{'timestamp': '1624616269792', 'hashtags': ['ENG', 'GER']}
607
{'timestamp': '1624616269804', 'hashtags': ['EURO2020', 'POR']}
608
{'timestamp': '1624616269983', 'hashtags': ['ManCity', 'EURO2020']}
609
{'timestamp': '1624616270289', 'hashtags': ['EURO2020', 'Heineken']}
610
611
{'timestamp': '1624616271271', 'hashtags': ['EURO2020']}
612
613
{'timestamp': '1624616271698', 'hashtags': ['EURO2020', '

{'timestamp': '1624616368374', 'hashtags': ['EURO2020', 'eng', 'football', 'samcurran']}
799
800
{'timestamp': '1624616369281', 'hashtags': ['EURO2020']}
801
{'timestamp': '1624616369471', 'hashtags': ['OTD', 'EURO2020', 'FRA']}
802
{'timestamp': '1624616370245', 'hashtags': ['FRA', 'SUI', 'CRO', 'ESP', 'ITA', 'AUT', 'BEL', 'POR']}
803
{'timestamp': '1624616371422', 'hashtags': ['EURO2020']}
804
{'timestamp': '1624616371813', 'hashtags': ['VamosEspaña', 'EURO2020']}
805
{'timestamp': '1624616371936', 'hashtags': ['EURO2020']}
806
807
{'timestamp': '1624616372489', 'hashtags': ['ManCity', 'EURO2020']}
808
809
810
{'timestamp': '1624616373611', 'hashtags': ['Finnische', 'Russland', 'Corona', 'EURO2020', 'Superspreader']}
811
{'timestamp': '1624616374042', 'hashtags': ['EURO2020']}
812
813
{'timestamp': '1624616377354', 'hashtags': ['EURO2020']}
814
815
{'timestamp': '1624616378432', 'hashtags': ['Heineken', 'musulmane']}
816
817
{'timestamp': '1624616378856', 'hashtags': ['ENG', 'GER']}


{'timestamp': '1624616465919', 'hashtags': ['POR']}
1008
1009
1010
{'timestamp': '1624616466264', 'hashtags': ['EURO2020']}
1011
{'timestamp': '1624616466498', 'hashtags': ['EURO2020']}
1012
1013
1014
{'timestamp': '1624616467600', 'hashtags': ['ManCity', 'EURO2020']}
1015
1016
1017
1018
{'timestamp': '1624616470992', 'hashtags': ['Euro2020']}
1019
{'timestamp': '1624616471509', 'hashtags': ['EURO2020']}
1020
{'timestamp': '1624616473081', 'hashtags': ['EURO2020']}
1021
1022
{'timestamp': '1624616473119', 'hashtags': ['EURO2020', 'BCH']}
1023
{'timestamp': '1624616473500', 'hashtags': ['EURO2020', 'ENG']}
1024
{'timestamp': '1624616474299', 'hashtags': ['EURO2020']}
1025
1026
1027
{'timestamp': '1624616475587', 'hashtags': ['EURO2020']}
1028
{'timestamp': '1624616475823', 'hashtags': ['EURO2020']}
1029
1030
{'timestamp': '1624616478173', 'hashtags': ['EURO2020', 'ENG']}
1031
{'timestamp': '1624616478394', 'hashtags': ['AskVizAnything']}
1032
1033
1034
{'timestamp': '1624616481217', 'ha

ProtocolError: ('Connection broken: IncompleteRead(0 bytes read)', IncompleteRead(0 bytes read))